## Load Conversations

In [1]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [2]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [3]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [4]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [5]:
# df.groupby("label").count()

In [6]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [7]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [8]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,319,319,319
2. Normal,1661,1661,1661
3. Not respect,364,364,364


In [9]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [10]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


In [11]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

In [14]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


In [49]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from collections import Counter

def print_accuracy(df, label_fn):
    true_labels = df["label"].values
    majority_class, count = Counter(true_labels).most_common()[0]
    predictions = [majority_class]*len(true_labels)
    
    true_labels_value = []
    predictions_value = []
    
    for l, p in zip(true_labels, predictions):
        if label_fn(l) is None:
            continue
            
        if label_fn(p) is None:
            print(p)
            
        true_labels_value.append(label_fn(l))
        predictions_value.append(label_fn(p))
    
    
    p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro', zero_division=0)
    metrics = {
        "rmse": np.sqrt(mean_squared_error(true_labels_value, predictions_value)),
        "r2": r2_score(true_labels_value, predictions_value),
        
        "accuracy": accuracy_score(true_labels, predictions),
        "f1": f1,
        "precision": p,
        "recall": r,
    }

    for k in metrics:
        print(f"{k}: {np.mean(metrics[k]):.3f} ± {np.std(metrics[k]):.3f}")
#     eval_metrics = {}
#     for i in tqdm(range(10)):
#         metrics = train_model(vocab, wc, labels)
#         for k in metrics:
#             if k not in eval_metrics:
#                 eval_metrics[k] = []
#             eval_metrics[k].append(metrics[k])

In [50]:
def closeness_to_vec1(label):
    if label == "1. Close":
        return 1
    elif label == "2. Know each other":
        return 0.5
    elif label == "3. Don't know each other": 
        return 0
    elif label == "4. Don't like each other":
        return None
    else:
        return None

def authority_to_vec1(label):
    if label == '0. Very respect':
        return 1
    elif label == '1. Respect': 
        return 0.5
    elif label == '2. Normal':
        return 0
    elif label == '3. Not respect':
        return None
    else:
        return None
    
def authority_to_vec2(label):
    if label == '0. Very respect':
        return None
    elif label == '1. Respect': 
        return 1
    elif label == '2. Normal':
        return 0.5
    elif label == '3. Not respect':
        return 0
    else:
        return None
    

In [51]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
print_accuracy(df, closeness_to_vec1)

Loaded 1234 records from ./Task1/annotated_conersations.jsonl
rmse: 0.636 ± 0.000
r2: -1.021 ± 0.000
accuracy: 0.451 ± 0.000
f1: 0.155 ± 0.000
precision: 0.113 ± 0.000
recall: 0.250 ± 0.000


In [52]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = [])
print_accuracy(df, closeness_to_vec1)

Loaded 2486 records from ./Task2/annotated/annotated.jsonl
rmse: 0.374 ± 0.000
r2: -0.228 ± 0.000
accuracy: 0.700 ± 0.000
f1: 0.206 ± 0.000
precision: 0.175 ± 0.000
recall: 0.250 ± 0.000


In [53]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = [])
print_accuracy(df, closeness_to_vec1)

Loaded 1221 records from ./Task3/annotated/annotated.jsonl
rmse: 0.326 ± 0.000
r2: -0.370 ± 0.000
accuracy: 0.570 ± 0.000
f1: 0.182 ± 0.000
precision: 0.143 ± 0.000
recall: 0.250 ± 0.000


In [54]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = [])
print_accuracy(df, authority_to_vec1)

Loaded 1234 records from ./Task1/annotated_conersations.jsonl
rmse: 0.512 ± 0.000
r2: -0.651 ± 0.000
accuracy: 0.557 ± 0.000
f1: 0.179 ± 0.000
precision: 0.139 ± 0.000
recall: 0.250 ± 0.000


In [55]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
print_accuracy(df, authority_to_vec2)

Loaded 2486 records from ./Task2/annotated/annotated.jsonl
rmse: 0.270 ± 0.000
r2: -0.001 ± 0.000
accuracy: 0.709 ± 0.000
f1: 0.276 ± 0.000
precision: 0.236 ± 0.000
recall: 0.333 ± 0.000


In [56]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
print_accuracy(df, authority_to_vec2)

Loaded 1221 records from ./Task3/annotated/annotated.jsonl
rmse: 0.187 ± 0.000
r2: -0.009 ± 0.000
accuracy: 0.861 ± 0.000
f1: 0.308 ± 0.000
precision: 0.287 ± 0.000
recall: 0.333 ± 0.000
